In [1]:
%run "wall_of_imports.ipynb"
%run "agent_state.ipynb"

In [2]:
### 4. Specialized Agents Implementation
def itinerary_agent(state: AgentState) -> dict:
    """Creates customized travel itineraries based on user preferences."""
    try:
        # First, retrieve relevant destination information
        rag_chain = setup_rag_chain()
        retrieval_result = rag_chain.invoke({"input": state.query})
        
        # Extract destination information from query using LLM
        extraction_prompt = ChatPromptTemplate.from_template("""Extract the key travel information from the user's query.
        Return a JSON object with these fields (leave empty if not mentioned):
        {{
            "destinations": ["list of mentioned destinations"],
            "duration": "total trip duration in days",
            "budget": "budget information",
            "interests": ["list of mentioned interests/activities"],
            "travel_dates": "approximate travel dates",
            "travelers": "number and type of travelers (family, couple, solo, etc.)"
        }}
        
        Query: {input}
        """)       
        
        extraction_chain = extraction_prompt | llm | StrOutputParser()

        try:
            # First check if the string is not empty
            if extraction_result := extraction_chain.invoke({"input": state.query}).strip():
                extracted_info = json.loads(extraction_result)
            else:
                extracted_info = {
                    "destinations": [],
                    "duration": "",
                    "budget": "",
                    "interests": [],
                    "travel_dates": "",
                    "travelers": ""
                }
            state.context.update(extracted_info)
        except json.JSONDecodeError:
            # Provide default structure when parsing fails
            extracted_info = {
                "destinations": [],
                "duration": "",
                "budget": "",
                "interests": [],
                "travel_dates": "",
                "travelers": ""
            }
            state.context.update(extracted_info)

        
        # Generate itinerary using retrieved information and extracted parameters
        itinerary_prompt = ChatPromptTemplate.from_template("""You are a travel itinerary expert. Create a detailed day-by-day travel itinerary
        based on the user's preferences and the retrieved destination information.
        
        For each day, include morning activities, lunch suggestions, afternoon activities, dinner recommendations, and 
        evening activities or relaxation options
        
        Also include practical advice about transportation between attractions, estimated costs, time management tips, and local customs.
        
        Make the itinerary realistic in terms of travel times and activities per day.
        
        Context information:
        {context_str}
        
        Extracted travel parameters:
        {parameters}
        
        Query: {input}
        """)        
        
        # Format the context and parameters for the prompt
        context_str = retrieval_result.get("answer", "")
        parameters_str = json.dumps(state.context, indent=2)
        
        itinerary_chain = itinerary_prompt | llm | StrOutputParser()
        response = itinerary_chain.invoke({
            "input": state.query,
            "context_str": context_str,
            "parameters": parameters_str
        })
        
        return {
            "agent_response": response,
            "context": state.context,
            "query": state.query
        }
        
    except Exception as e:
        return {
            "error": str(e),
            "context": state.context,
            "query": state.query
        }